In [8]:
# Descripción breve del programa: Este programa simula un robot de limpieza en un entorno de cuadrícula.
# Autores: José Antonio Moreno Tahuilan A01747922
#          Ángel Armando Márquez Curiel A01750147
# Fecha de creación/modificación: Fecha


import mesa
import random
import numpy as np
from mesa.experimental import JupyterViz
import solara
from matplotlib.figure import Figure
import time

DIRTY = set()
TIME_LIMIT = 25

class RobotAgent(mesa.Agent):
    """A cleaning robot agent which can move and clean dirty cells."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.steps = 0

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)
        self.steps += 1

    def remove_agent(self, agent):
        self.model.schedule.remove(agent)

    def clean(self):
        x, y = self.pos
        a = (x,y)
        if a in DIRTY:
            DIRTY.remove(a)
            cell = self.model.grid.get_cell_list_contents([self.pos])
            dirty_cell = [obj for obj in cell if isinstance(obj, DirtyCell)]
            if dirty_cell:
                self.model.schedule.remove(dirty_cell[0])
                self.model.grid.remove_agent(dirty_cell[0])

    def step(self):
        self.move()
        self.clean()

class DirtyCell(mesa.Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def step(self):
        ...

class Model(mesa.Model):
    def __init__(self, num_agents, grid_M, grid_N, umbral):
        """A cleaning environment model which contains cleaning robots and dirty cells."""
        self.num_agents = num_agents
        self.grid = mesa.space.MultiGrid(grid_M, grid_N, True)
        self.schedule = mesa.time.RandomActivation(self)
        self.actual_time = 0
        
        count = num_agents + 1
        self.start_time = time.time()
        self.grid_M = grid_M
        self.grid_N = grid_N

        for i in range(self.num_agents):
            a = RobotAgent(i, self)
            self.schedule.add(a)
            self.grid.place_agent(a, (0, 0))

        for row in range(grid_M):
            for column in range(grid_N):
                random_number = random.randint(1, 100)
                if (random_number < umbral):
                    DIRTY.add((row,column))
                    a = DirtyCell(count,self)
                    self.schedule.add(a)
                    self.grid.place_agent(a, (row,column))
                    count += 1
       
        self.running = True

    def step(self):
        self.actual_time = time.time() - self.start_time
        self.schedule.step()
        if len(DIRTY) == 0 or time.time() - self.start_time > TIME_LIMIT:
            self.running = False

model = Model(3, 10, 10, 50)
model.step()
while len(DIRTY) > 0:
    model.step()

def make_histogram(model):
    # Note: you must initialize a figure using this method instead of
    # plt.figure(), for thread safety purpose
    fig = Figure()
    ax = fig.subplots()
    stp = []
    for agent in model.schedule.agents:
        if isinstance(agent, RobotAgent):
            stp.append(agent.steps)
    categories = ['Steps']
    frequencies = [stp[0] * len(stp)]
    ax.barh(categories, frequencies)
    solara.FigureMatplotlib(fig)

def make_cleanANDdirty(model):
    fig = Figure()
    ax = fig.subplots()
    dirtyones = len(DIRTY)
    cleanones = model.grid_M * model.grid_N - dirtyones
    categories = ['Dirty', 'Clean']
    frequencies = [dirtyones, cleanones]
    ax.barh(categories, frequencies)
    solara.FigureMatplotlib(fig)

def make_time(model):
    fig = Figure()
    ax = fig.subplots()
    time = [model.actual_time]
    categories = ['Time']
    frequencies = [time[0]]
    ax.barh(categories, frequencies)
    solara.FigureMatplotlib(fig)



def agent_portrayal(agent):
    if isinstance(agent, RobotAgent):
        return {
            "color": "tab:blue",
            "size": 50,
        }
    elif isinstance(agent, DirtyCell):
        return {
            "color": "tab:red",
            "size": 50,
        }


model_params = {
    "num_agents": {
        "type": "SliderInt",
        "value": 50,
        "label": "Number of agents:",
        "min": 1,
        "max": 30,
        "step": 1,
    },
    "grid_M": 10,
    "grid_N": 10,
    "umbral": {
        "type": "SliderInt",
        "value": 80,
        "label": "Umbral:",
        "min": 10,
        "max": 100,
        "step": 1,
    }
}

page = JupyterViz(
    Model,
    model_params,
    measures=[make_histogram, make_cleanANDdirty, make_time],
    name="Cleaning Environment",
    agent_portrayal=agent_portrayal
    
)
# This is required to render the visualization in the Jupyter notebook
page

Cannot show ipywidgets in text